## Reading in data available through Maize GDB (Maize Genetics and Genomics Database)
The purpose of this notebook is to read in and do a preliminary analysis of the data related to text descriptions that are available through Maize GDB. The data was provided in the form of the input file by a request through Maize GDB curators, rather than obtained through an already available file from the database. The data needs to be organized and also restructured into a standard format that will allow it to be easily combined with datasets from other resources. This notebook takes the following input files that were obtained from MaizeGDB and produces a set of files that have standard columns that are listed and described below.

### Files read
```
plant-data/databases/maizegdb/pheno_genes.txt
plant-data/databases/maizegdb/maize_v3.gold.gaf
```


### Files created
```
plant-data/reshaped_data/maizegdb_phenotype_descriptions.csv
plant-data/reshaped_data/maizegdb_go_annotations.csv
```

### Columns in the created files
* **species_name**: String is the name of the species.
* **species_code**: String identifier for the species, uses the 3-letter codes from KEGG.
* **unique_gene_identifiers**: Pipe delimited list of gene identifers, names, models, etc that uniquely refer to this gene.
* **other_gene_identifiers**: Same as the previous, but may not uniquely refer to a given gene.
* **gene_models**: Pipe delimited list of gene model names, subset of unique_gene_identifiers.
* **text_unprocessed**: A free text field for any descriptions of phenotyes associated with this gene.
* **annotations**: Pipe delimited list of gene ontology term identifiers.
* **reference_name**: String naming the database or paper that was the source for this data.
* **reference_link**: The link to the reference resource if applicable.
* **reference_file**: The specific name of the file from which this data comes if applicable.

In [1]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import sys
import os
import warnings
import pandas as pd
import numpy as np
import itertools
import re
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

sys.path.append("../utils")
from constants import NCBI_TAG, UNIPROT_TAG, EVIDENCE_CODES

sys.path.append("../../oats")
from oats.nlp.preprocess import concatenate_with_delim, subtract_string_lists, replace_delimiter, concatenate_texts
from oats.nlp.small import remove_punctuation, remove_enclosing_brackets, add_prefix_safely

OUTPUT_DIR = "../reshaped_data"
mpl.rcParams["figure.dpi"] = 200
warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

[nltk_data] Downloading package punkt to /home/ian/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Columns that should be in the final reshaped files.
reshaped_columns = [
 "species_name",
 "species_code",
 "unique_gene_identifiers", 
 "other_gene_identifiers", 
 "gene_models", 
 "text_unprocessed", 
 "annotations", 
 "reference_name",
 "reference_link",
 "reference_file"]

# Creating and testing a lambda for finding gene model strings.
gene_model_pattern_1 = re.compile("grmzm.+")
gene_model_pattern_2 = re.compile("zm[0-9]+d[0-9]+")
is_gene_model = lambda s: bool(gene_model_pattern_1.match(s.lower() or gene_model_pattern_2.match(s.lower())))

### File with genes and phenotype descriptions (pheno_genes.txt)
Note that fillna is being used here to replace missing values with an empty string. This is done so that the missing string will be quantified when checking for the number of occurences of unique values from different columns, see the analysis below. However this is not necessary as a preprocessing step because when the data is read in and appended to a dataset object later, any missing values or empty strings will be handled at that step.

In [3]:
filename = "../databases/maizegdb/pheno_genes.txt"
usecols = ["phenotype_name", "phenotype_description", "locus_name", "alleles", "locus_synonyms", "v3_gene_model", "v4_gene_model", "uniprot_id", "ncbi_gene"]
df = pd.read_table(filename, usecols=usecols)
df.fillna("", inplace=True)
print(df[["phenotype_name","phenotype_description"]].head(10))
print(df.shape)

        phenotype_name                              phenotype_description
0     2-seeded kernels                                                   
1   A1 null transcript  No color in the aleurone, specifically no colo...
2    aberrant seedling  first leaf is small, round and flat; second le...
3    aberrant seedling  first leaf is small, round and flat; second le...
4  abnormal root hairs                        root hairs fail to elongate
5  abnormal root hairs                        root hairs fail to elongate
6  abnormal root hairs                        root hairs fail to elongate
7  abnormal root hairs                        root hairs fail to elongate
8  abnormal root hairs                        root hairs fail to elongate
9  abnormal root hairs                        root hairs fail to elongate
(3616, 9)


In [4]:
df

,phenotype_name,phenotype_description,locus_name,alleles,locus_synonyms,v3_gene_model,v4_gene_model,uniprot_id,ncbi_gene
0,2-seeded kernels,,wcr1,Wcr1-reference,wandering carpel1|wcr1,,,,
1,A1 null transcript,"No color in the aleurone, specifically no colo...",a1,a1-m2-8004::dSpm,a1|anthocyaninless1|bnl(a1)|DFR|FNR|gsy38(A1)|...,GRMZM2G026930,Zm00001d044122,P51108,100286107
2,aberrant seedling,"first leaf is small, round and flat; second le...",pve1,pve1-M2|pve1-R,AC211276.4_FG008|cl12053_1|cl12053_1(383)|inve...,AC211276.4_FG008,Zm00001d013672,,103626037
3,aberrant seedling,"first leaf is small, round and flat; second le...",ubl1,ubl1-1,si945031h05(438)|si945031h05a|ubl1,GRMZM2G156575,Zm00001d017432,,100275088
4,abnormal root hairs,root hairs fail to elongate,bhlh10,bhlh10-1|bhlh10-2,Lotus japonicus roothairless1-like|lrl5|transc...,GRMZM2G067654,Zm00001d045107,,103637976
...,...,...,...,...,...,...,...,...,...
3611,zebra necrotic leaf,Necrotic tissue appears between veins in regul...,zb1,zb1,zb1|zebra crossbands1,,,,
3612,zebra necrotic leaf,Necrotic tissue appears between veins in regul...,zn1,zn1|zn1-N25,zebra necrotic1|zn1,,,,
3613,zebra necrotic leaf,Necrotic tissue appears between veins in regul...,zn2,zn2|zn2-4-6(4461)|zn2-56-3012-10|zn2-94-234|zn...,zebra necrotic2|zn2,,,,
3614,zebra necrotic leaf,Necrotic tissue appears between veins in regul...,zn*-N571D,zn*-N571D,zebra necroticN571D|zn*-571D|zn*-N571D,,,,


Text information about the phenotypes are contained in both the phenotype name and phenotype description for these data. The can be concatenated and retained together in a new description column that contains all this information, or just the phenotype description could be retained, depending on which data should be used downstream for making similarity comparisons. This is different than for most of the other sources of text used. The next cell looks at how many unique values there are in this data for each column.

In [5]:
# Finding out how many unique values there are for each column.
unique_values = {col:len(pd.unique(df[col].values)) for col in df.columns}
for k,v in unique_values.items():
    print("{:24}{:8}".format(k,v))

phenotype_name               648
phenotype_description        379
locus_name                  1410
alleles                     2088
locus_synonyms              1408
v3_gene_model                482
v4_gene_model                469
uniprot_id                   140
ncbi_gene                    503


There are a fairly small number of distinct phenotype descriptions (379) compared to the number of lines that are in the complete dataset (3,616). This means that the same descriptions is occuring many times. Look at which descriptions are occuring most often.

In [6]:
# Get a list sorted by number of occurences for each phenotype description.
description_counts = df["phenotype_description"].value_counts().to_dict()
sorted_tuples = sorted(description_counts.items(), key = lambda x: x[1], reverse=True)
for t in sorted_tuples[0:10]:
    print("{:6}    {:20}".format(t[1],t[0][:70]))

   892                        
    90    Anthers shriveled, not usually exerted from glum. Pollen absent or abo
    87    seedling white, yellow or palegreen, becomes green, often normal
    81    Two classes of albino seedlings: Class I characterized by white or pal
    81    Seedling leaves are white.
    77    endosperm is opaque and firm, not chalky and not waxy
    73    small kernel is the consistent characteristic but variable in other as
    73    a general term describing an improperly developed endosperm that appea
    71    endosperm has a soft, chalk-like texture, usually a reduced yellow col
    65    lighter green seedlings or plants; less yellow than yellow green


The only description that occurs far more often than the next is an empty string, where this information is missing entirely. The next cell looks at how many phrases are included in the phenotype description values. Most have a single phrase, some have multiple. These look like they are mainly separated with semicolons.

In [7]:
# Plotting distributions of number of phrases in each description.
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.set_title("Phenotype Descriptions")
ax2.set_title("Phenotype Descriptions")
ax1.set_xlabel("Number of phrases")
ax2.set_xlabel("Number of words")
x1 = [len(sent_tokenize(x)) for x in df["phenotype_description"].values]
x2 = [len(word_tokenize(x)) for x in df["phenotype_description"].values]
ax1.hist(x1, bins=15, range=(0,15), density=False, alpha=0.8, histtype='stepfilled', color="black", edgecolor='none')
ax2.hist(x2, bins=30, range=(0,150), density=False, alpha=0.8, histtype='stepfilled', color="black", edgecolor='none')
fig.set_size_inches(15,4)
fig.tight_layout()
fig.show()
plt.close()

In [8]:
# Restructuring the dataset to include all the expected column names.
combine_gene_columns = lambda row, columns: concatenate_with_delim("|", [row[column] for column in columns])
combine_text_columns = lambda row, columns: concatenate_with_delim("; ", [row[column] for column in columns])
df["text_unprocessed"] = df.apply(lambda x: combine_text_columns(x, ["phenotype_name", "phenotype_description"]), axis=1)
df["uniprot_id"] = df["uniprot_id"].apply(add_prefix_safely, prefix=UNIPROT_TAG)
df["ncbi_gene"] = df["ncbi_gene"].apply(add_prefix_safely, prefix=NCBI_TAG)
df["unique_gene_identifiers"] = df.apply(lambda x: combine_gene_columns(x, ["locus_name", "alleles", "v3_gene_model", "v4_gene_model", "uniprot_id", "ncbi_gene"]), axis=1)
df["other_gene_identifiers"] = df["locus_synonyms"]
df["gene_models"] = df.apply(lambda x: combine_gene_columns(x, ["v3_gene_model", "v4_gene_model"]), axis=1)
df["species_name"] = "maize"
df["species_code"] = "zma"
df["annotations"] = ""
df["reference_name"] = "MaizeGDB"
df["reference_link"] = "https://www.maizegdb.org/"
df["reference_file"] = "pheno_genes.txt"
df["other_gene_identifiers"] = df.apply(lambda row: subtract_string_lists("|", row["other_gene_identifiers"],row["unique_gene_identifiers"]), axis=1)
df = df[reshaped_columns]
df.head()

,species_name,species_code,unique_gene_identifiers,other_gene_identifiers,gene_models,text_unprocessed,annotations,reference_name,reference_link,reference_file
0,maize,zma,wcr1|Wcr1-reference,wandering carpel1,,2-seeded kernels,,MaizeGDB,https://www.maizegdb.org/,pheno_genes.txt
1,maize,zma,a1|a1-m2-8004::dSpm|GRMZM2G026930|Zm00001d0441...,anthocyaninless1|bnl(a1)|DFR|FNR|gsy38(A1)|npi...,GRMZM2G026930|Zm00001d044122,"A1 null transcript; No color in the aleurone, ...",,MaizeGDB,https://www.maizegdb.org/,pheno_genes.txt
2,maize,zma,pve1|pve1-M2|pve1-R|AC211276.4_FG008|Zm00001d0...,cl12053_1|cl12053_1(383)|inverted formin-2-lik...,AC211276.4_FG008|Zm00001d013672,"aberrant seedling; first leaf is small, round ...",,MaizeGDB,https://www.maizegdb.org/,pheno_genes.txt
3,maize,zma,ubl1|ubl1-1|GRMZM2G156575|Zm00001d017432|ncbi=...,si945031h05(438)|si945031h05a,GRMZM2G156575|Zm00001d017432,"aberrant seedling; first leaf is small, round ...",,MaizeGDB,https://www.maizegdb.org/,pheno_genes.txt
4,maize,zma,bhlh10|bhlh10-1|bhlh10-2|GRMZM2G067654|Zm00001...,Lotus japonicus roothairless1-like|lrl5|transc...,GRMZM2G067654|Zm00001d045107,abnormal root hairs; root hairs fail to elongate,,MaizeGDB,https://www.maizegdb.org/,pheno_genes.txt


In [9]:
df["text_unprocessed"].values[:1000]

array(['2-seeded kernels',
       'A1 null transcript; No color in the aleurone, specifically no color due to anthocyanins.  In addition, lack of a1 transcript is confirmed.',
       'aberrant seedling; first leaf is small, round and flat; second leaf is rolled, bent and weak.  Leaf tissue is thin, weak and yellow green.  Succeeding leaves fail to develop and seedling dies.',
       'aberrant seedling; first leaf is small, round and flat; second leaf is rolled, bent and weak.  Leaf tissue is thin, weak and yellow green.  Succeeding leaves fail to develop and seedling dies.',
       'abnormal root hairs; root hairs fail to elongate',
       'abnormal root hairs; root hairs fail to elongate',
       'abnormal root hairs; root hairs fail to elongate',
       'abnormal root hairs; root hairs fail to elongate',
       'abnormal root hairs; root hairs fail to elongate',
       'abnormal root hairs; root hairs fail to elongate',
       'abnormal stomates; Guard cells development is abnormal, 

In [10]:
# Outputting the dataset of descriptions to a csv file.
path = os.path.join(OUTPUT_DIR,"maizegdb_phenotype_descriptions.csv")
df.to_csv(path, index=False)

### File with high confidence gene ontology annotations (maize_v3.gold.gaf)
This file was generated as part of the [Maize GAMER](https://onlinelibrary.wiley.com/doi/full/10.1002/pld3.52)  publication (Wimalanathan et al., 2018). The annotations include all of the associations between maize genes and ontology terms from GO where the terms have been experimentally confirmed to represent correct functional annotations for those genes.

In [11]:
filename = "../databases/maizegdb/maize_v3.gold.gaf"
df = pd.read_table(filename, skiprows=1)
df.fillna("", inplace=True)
df.head()

,!db,db_object_id,db_object_symbol,qualifier,term_accession,db_reference,evidence_code,with,aspect,db_object_name,db_object_synonym,db_object_type,taxon,date,assigned_by,annotation_extension,gene_product_form_id
0,MaizeGDB,bx1,GRMZM2G085381,,GO:0000162,PMID:9235894,IMP,,P,benzoxazinless1,MaizeGDB:GRMZM2G085381,protein,taxon:4577,20151010,MaizeGDB,,
1,MaizeGDB,mpk2,GRMZM2G020216,,GO:0000302,PMID:20693409,IDA,,P,MAP kinase2,MaizeGDB:GRMZM2G020216,protein,taxon:4577,20151010,MaizeGDB,,
2,MaizeGDB,crs1,GRMZM2G078412,,GO:0000373,PMID:15598799,IDA,,P,chloroplast RNA splicing1,MaizeGDB:GRMZM2G078412,protein,taxon:4577,20151010,MaizeGDB,,
3,MaizeGDB,crs2,GRMZM2G132021,,GO:0000373,PMID:11179231,IDA,,P,chloroplast RNA splicing2,MaizeGDB:GRMZM2G132021,protein,taxon:4577,20151010,MaizeGDB,,
4,MaizeGDB,gams1,ga-ms1,,GO:0000775,MaizeGDB:123623,IMP,,C,gametophytic male sterile1,MaizeGDB:ga-ms1,protein,taxon:4577,20151010,MaizeGDB,,


In [13]:
# Restructuring the dataset to include all the expected column names.
df["text_unprocessed"] = ""
df["unique_gene_identifiers"] = df.apply(lambda x: combine_gene_columns(x, ["db_object_id", "db_object_symbol"]), axis=1)
df["other_gene_identifiers"] = df.apply(lambda x: combine_gene_columns(x, ["db_object_name", "db_object_synonym"]), axis=1)
df["gene_models"] =  df["unique_gene_identifiers"].map(lambda x: "".join([s for s in x.split("|") if is_gene_model(s)]))
df["species_name"] = "maize"
df["species_code"] = "zma"
df["annotations"] = df["term_accession"]
df["reference_name"] = "MaizeGDB"
df["reference_link"] = "https://www.maizegdb.org/"
df["reference_file"] = "pheno_genes.txt"
df = df[reshaped_columns]
df.head()

,species_name,species_code,unique_gene_identifiers,other_gene_identifiers,gene_models,text_unprocessed,annotations,reference_name,reference_link,reference_file
0,maize,zma,bx1|GRMZM2G085381,benzoxazinless1|MaizeGDB:GRMZM2G085381,GRMZM2G085381,,GO:0000162,MaizeGDB,https://www.maizegdb.org/,pheno_genes.txt
1,maize,zma,mpk2|GRMZM2G020216,MAP kinase2|MaizeGDB:GRMZM2G020216,GRMZM2G020216,,GO:0000302,MaizeGDB,https://www.maizegdb.org/,pheno_genes.txt
2,maize,zma,crs1|GRMZM2G078412,chloroplast RNA splicing1|MaizeGDB:GRMZM2G078412,GRMZM2G078412,,GO:0000373,MaizeGDB,https://www.maizegdb.org/,pheno_genes.txt
3,maize,zma,crs2|GRMZM2G132021,chloroplast RNA splicing2|MaizeGDB:GRMZM2G132021,GRMZM2G132021,,GO:0000373,MaizeGDB,https://www.maizegdb.org/,pheno_genes.txt
4,maize,zma,gams1|ga-ms1,gametophytic male sterile1|MaizeGDB:ga-ms1,,,GO:0000775,MaizeGDB,https://www.maizegdb.org/,pheno_genes.txt


In [14]:
# Outputting the dataset of annotations to a csv file.
path = os.path.join(OUTPUT_DIR,"maizegdb_curated_go_annotations.csv")
df.to_csv(path, index=False)